In [1]:
#!g1.1
%pip install transformers sentencepiece pympi-ling razdel

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [39]:
#!g1.1
!/usr/local/bin/python3 -m pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 2.1 MB 1.2 MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [41]:
#!g1.1
%pip install pytorch_lightning

Defaulting to user installation because normal site-packages is not writeable
  Using cached pytorch_lightning-1.6.0-py3-none-any.whl (582 kB)


In [1]:
#!g1.1
import os, sys
import re, random
import copy
import json
import pympi
import pymystem3
import pandas as pd
import numpy as np
from pprint import pprint
from itertools import chain
from tqdm import tqdm

sys.path.append('../')
from pyeaf.pyeaf import EAFReader
from pyeaf.text import RSLStemmer, GramBinarizer

In [2]:
#!g1.1
#er = EAFReader(directory="../Разметки eaf")
#er.load()

In [3]:
#!g1.1
#sentences_rus = er.get_sentences(er.RUS)
#sentences_rsl = er.get_sentences(er.RSL_R)

# sentences_rsl_left = er.get_sentences(er.RSL_L)

In [87]:
#!g1.1
#stem_sentences_rsl = list(map(RSLStemmer.stem_sentence, sentences_rsl))

In [5]:
#!g1.1
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import textwrap
from torch.nn import functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Russian language should be tokenized the same way it did in the original model:

In [6]:
#!g1.1
from transformers import T5Tokenizer, MT5Model, AdamW

# load model
tokenizer = T5Tokenizer.from_pretrained("cointegrated/rut5-small")

# test
tokenizer('Какой-то текст на русском языке.')

{'input_ids': [3835, 817, 264, 832, 5912, 310, 7451, 637, 5640, 324, 260, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [4]:
#!g1.1
# don't mind me just looking
tokenizer.vocab_size

20200

Теперь мы можем настроить логгирование и загрузить модельку для русского вот отсюда:

https://huggingface.co/cointegrated/rut5-small

Я беру именно эту, потому что там только русский язык и моделька маленькая, значит, моя память это потянет.

In [7]:
#!g1.1
import logging
from transformers import MT5ForConditionalGeneration

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model = MT5ForConditionalGeneration.from_pretrained("cointegrated/rut5-small", return_dict=True).to(device)

In [ ]:
#!g1.1


In [6]:
#!g1.1
model.config

MT5Config {
  "_name_or_path": "cointegrated/rut5-small",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.9.2",
  "use_cache": true,
  "vocab_size": 20100
}

In [5]:
#!g1.1
# test
input_ids_test = tokenizer(
    "Что тут вообще происходит?",
    return_tensors="pt"
).input_ids.to(device)

generated_ids = model.generate(input_ids_test)

In [6]:
#!g1.1
# test
preds = [
    tokenizer.decode(input_id)
    for input_id in generated_ids
]

' '.join(preds)

'<pad> Что тут вообще происходит вообще</s>'

In [7]:
#!g1.1
# don't mind me just checking
text = 'Ехал Грека через реку, видит Грека в реке рак. '
inputs = tokenizer(text, return_tensors='pt').to(device)
with torch.no_grad():
    hypotheses = model.generate(
        **inputs, 
        do_sample=True, top_p=0.95, num_return_sequences=10, 
        repetition_penalty=2.5,
        max_length=32,
    )
for h in hypotheses:
    print(tokenizer.decode(h, skip_special_tokens=True))

Эхал Грека поблизилась от реки, видит стела неизвестный греки в Раком.
Грека в реке Лехаре увидела смертника грекой по иску от Рака, видит Ехал Греку.
Ехал Грека после расследование крушения у Трампа и видит греку рак в реке.
Грека и Ехал греки под рекой сошла на место, видит Траку в проливе Рак.
Грека и Ехал греко, видевшие погибшую в Ракке острова Трампа через реку вылетел в древнюю
Грека на борту через реку, видит греков в реке рак, замечает Ехал Хурга.
Грека выбросит Ехал Грек через реку, видят Авра и врач Афганистана.
Грека в реке, которая перебросила Рим через реку подряд видит греко на рак.
Ехал Грека для проведения корабля у отрезания в реку Рима нашли рак по "кору" с Афганистана
Грека приводит в реке рак, видит погибшую грек во время столкновения с Пелом.


In [8]:
#!g1.1
# test
batch_sentences_ru = [["hello", "rus", "sentence"]]
batch_sentences_rsl = [["hello", "rsl", "sentence"]]
encoded_inputs = tokenizer(batch_sentences_ru,
                           batch_sentences_rsl,
                           is_split_into_words=True)
print(encoded_inputs)

{'input_ids': [[790, 1578, 268, 259, 737, 263, 3060, 687, 541, 1, 790, 1578, 268, 259, 286, 263, 280, 3060, 687, 541, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [9]:
#!g1.1
# test
preds = [
    tokenizer.decode(input_id)
    for input_id in encoded_inputs['input_ids']
]

' '.join(preds)

'hello rus sentence</s> hello rsl sentence</s>'

### Подгрузим данные:

In [4]:
#!g1.1
generated_data = pd.read_csv('generated_stemed.csv', encoding='utf-8')

generated_data['stem_rus'] = generated_data['new_stem_sent_rus'].apply(lambda sent: sent.strip('[]\'').split('\', \''))
generated_data['rsl'] = generated_data['new_stem_rsl'].apply(lambda sent: sent.strip('[]\'').split('\', \''))
generated_data['gram_rus'] = generated_data['new_gram_sent_rus'].apply(lambda sent: json.loads(sent.replace('\'', '\"')))

NameError: name 'pd' is not defined

In [ ]:
#!g1.1

test_data = pd.read_csv('test_data.csv', encoding='utf-8')

test_data['stem_rus'] = test_data['test_stem_rus'].apply(lambda sent: sent.strip('[]\'').split('\', \''))
test_data['rsl'] = test_data['test_rsl'].apply(lambda sent: sent.strip('[]\'').split('\', \''))
test_data['gram_rus'] = test_data['test_gram_rus'].apply(lambda sent: json.loads(sent.replace('\'', '\"')))

In [ ]:
#!g1.1
generated_data.head(2)

In [ ]:
#!g1.1
class Preprocess:
    def __init__(self, tokenizer, max_length = 396):
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    #def list_of_lists(self, data_col):
    #    return list(map(list, data_col))
    
    def tokenization(self, batch):
        #mapped_rus = self.list_of_lists(batch['stem_rus'])
        #mapped_rsl = self.list_of_lists(batch['rsl'])
        model_inputs = self.tokenizer(batch['stem_rus'],
                                      max_length=self.max_length,
                                      padding="max_length",
                                      is_split_into_words=True,
                                      truncation="longest_first",
                                      add_special_tokens=True,
                                      return_tensors="pt")
        ## Setup the tokenizer for targets
        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(batch['rsl'],
                                    padding="max_length",
                                    truncation="longest_first",
                                    max_length=self.max_length,
                                    add_special_tokens=True,
                                    is_split_into_words=True,
                                    return_tensors="pt")
        model_inputs["labels"] = labels["input_ids"]
        
        return model_inputs
    
# batched=True, batch_size=None ?

p = Preprocess(tokenizer, max_length=396)
#train_dataset_encoded = p.tokenization(generated_data)
train_dataset_encoded = p.tokenization(test_data.iloc[0])  # test

In [26]:
#!g1.1
test_data.iloc[0]

test_stem_rus    ['а', 'когда', 'дед', 'мороз', 'вернуться', 'д...
test_gram_rus    [['CONJ'], ['CONJ'], ['им', 'мн', 'S', 'муж', ...
test_rsl         ['indx', 'дед.мороз', 'clf', 'clf.группа', 'cl...
stem_rus         [а, когда, дед, мороз, вернуться, домой, мален...
rsl              [indx, дед.мороз, clf, clf.группа, clf.перемес...
gram_rus         [[CONJ], [CONJ], [им, мн, S, муж, од], [им, мн...
Name: 0, dtype: object

In [ ]:
#!g1.1
# test 
tokenizer.decode(train_dataset_encoded['labels'].squeeze()) 

In [15]:
#!g1.1
train_dataset_encoded['input_ids'].shape[0] #[0], test_data['rsl'][0]

1

In [16]:
#!g1.1
# test
test_dataset_encoded = p.tokenization(test_data.iloc[0])

In [17]:
#!g1.1
train_dataset_encoded.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

> labels (torch.LongTensor of shape (batch_size,), optional) – Labels for computing the sequence classification/regression loss. Indices should be in [-100, 0, ..., config.vocab_size - 1]. All labels set to -100 are ignored (masked), the loss is only computed for labels in [0, ..., config.vocab_size]

In [66]:
#!g1.1
# del padding indeces from forward function
test_dataset_encoded['labels'][test_dataset_encoded['labels'] == 0] = -100

In [68]:
#!g1.1
# del padding indeces from forward function
train_dataset_encoded['labels'][train_dataset_encoded['labels'] == 0] = -100

In [67]:
#!g1.1
# test
test_dataset_encoded['labels'][0]

tensor([ 4578,   329, 13684,   260, 19625,   317,   280,   367,   317,   280,
          367,   260, 11963,   317,   280,   367,   260,  8628,  9522,  1334,
         5804,   817,   317,   280,   367,   317,   280,   367,   260, 11963,
          317,   280,   367,   260,  8628,  9522,  1334, 13684,   260, 19625,
          259,  6821,  1008, 11293,  6948,   805,  6722,  1182, 14518,  5494,
         1626,   333,   325,   263,  3933,  6918,  6426,  1358,   317,   280,
          367,   317,   280,   367,   260,  8628,  9209,  5021,     1,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100, 

In [10]:
#!g1.1
class MTDataset(Dataset):
    def __init__(self, data: pd.DataFrame, tokenizer: T5Tokenizer, max_length: int = 396):
        self.p = Preprocess(tokenizer, max_length=max_length)
        self.tokenizer = tokenizer
        self.data = data
        self.max_length = max_length
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]
        #print(self.max_length)
        #print(self.tokenizer)
        dataset_encoded = self.p.tokenization(data_row)
        dataset_encoded['labels'][dataset_encoded['labels'] == 0] = -100
        return dict(
            #rus=data_row['stem_rus'],
            #rsl=data_row['rsl'],
            input_ids=dataset_encoded['input_ids'].flatten(),
            attention_mask=dataset_encoded['attention_mask'].flatten(),
            labels=dataset_encoded['labels'].flatten()
        )

In [12]:
#!g1.1
tokenizer

PreTrainedTokenizer(name_or_path='cointegrated/rut5-small', vocab_size=20200, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'})

In [11]:
#!g1.1
#train_dataset = MTDataset(generated_data, tokenizer)
test_dataset = MTDataset(test_data, tokenizer)

In [12]:
#!g1.1
#test
for el in test_dataset:
    print(el)
    break

{'input_ids': tensor([  259,   308,   259,  5188, 13684, 14093,  5484, 11630,  5804,   817,
          259,  6821,  1008, 13684, 14093,   805,  6722,   778,  5494,  1626,
          259,   279,   388,  6024,   259,   308,   259,   396,  1049,   259,
         8052,   315,  6918,  1049,  1108,  6426,  1358, 17091,  6577,     1,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

In [13]:
#!g1.1
class MTDataModule(pl.LightningModule):
    def __init__(
        self,
        train_df: pd.DataFrame,
        test_df: pd.DataFrame,
        tokenizer: T5Tokenizer,
        batch_size: int = 16,
        max_length: int = 396
    ):
        super().__init__()
        self.batch_size = batch_size
        self.max_length = max_length
        self.train_df = train_df
        self.test_df = train_df
        self.tokenizer = tokenizer
    
    def setup(self):
        self.train_dataset = MTDataset(self.train_df, self.tokenizer, self.max_length)
        self.test_dataset = MTDataset(self.test_df, self.tokenizer, self.max_length)
        #for el in self.train_dataset:
        #    print(el)
        #    break
        
    def train_dataloader(self):
        #for el in self.train_dataset:
        #    print(el)
        #    break
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=8
        )
    
    def val_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            num_workers=8
        )
    
    def test_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            num_workers=8
        )
        

In [14]:
#!g1.1
BATCH_SIZE = 16
N_EPOCH = 3

data_module = MTDataModule(generated_data, test_data, tokenizer, batch_size=BATCH_SIZE, max_length=396)
data_module.setup()

In [15]:
#!g1.1
# test
data_module.train_dataloader()

In [24]:
#!g1.1
#%pip install pillow==8.2.0

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 8.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.7.0 requires torch==1.6.0, but you have torch 1.9.0 which is incompatible.
mmdet 2.3.0rc0+c6b5ca2 requires Pillow<=6.2.2, but you have pillow 8.2.0 which is incompatible.
mmdet 2.3.0rc0+c6b5ca2 requires torch==1.6.0, but you have torch 1.9.0 which is incompatible.
enot-utils 1.0.2 requires torch==1.6.0, but you have torch 1.9.0 which is incompatible.
enot-utils 1.0.2 requires tqdm>=4.50.0, but you have tqdm 4.49.0 which is incompatible.


In [16]:
#!g1.1
class RusRSLModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = MT5ForConditionalGeneration.from_pretrained(
            "cointegrated/rut5-small", return_dict=True).to(device)
    
    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        return output.loss, output.logits
    
    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        loss, outputs = self.forward(input_ids, attention_mask, labels)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        #print(batch_idx)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        loss, outputs = self.forward(input_ids, attention_mask, labels)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        loss, outputs = self.forward(input_ids, attention_mask, labels)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss
    
    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=0.0001)

In [17]:
#!g1.1
model = RusRSLModel().to(device)

In [28]:
#!g1.1
# test
model

RusRSLModel(
  (model): MT5ForConditionalGeneration(
    (shared): Embedding(20100, 512)
    (encoder): T5Stack(
      (embed_tokens): Embedding(20100, 512)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=512, out_features=384, bias=False)
                (k): Linear(in_features=512, out_features=384, bias=False)
                (v): Linear(in_features=512, out_features=384, bias=False)
                (o): Linear(in_features=384, out_features=512, bias=False)
                (relative_attention_bias): Embedding(32, 6)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedGeluDense(
                (wi_0): Linear(in_features=512, out_features=1024, bias=False)
                (wi_1): Linear(

In [ ]:
#!g1.1


In [18]:
#!g1.1
trainer = pl.Trainer(
    max_epochs=N_EPOCH,
    accelerator="gpu",
    devices=1
    #callbacks=pl.callbacks.progress.TQDMProgressBar(refresh_rate=30)
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [19]:
#!g1.1
%tensorboard --logdir logs/fit

Launching TensorBoard...


In [20]:
#!g1.1
trainer.fit(model, data_module.train_dataloader(), data_module.val_dataloader())

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                        | Params
------------------------------------------------------
0 | model | MT5ForConditionalGeneration | 64.6 M
------------------------------------------------------
64.6 M    Trainable params
0         Non-trainable params
64.6 M    Total params
258.578   Total estimated model params size (MB)
/home/jupyter/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/home/jupyter/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a b

[0;31mKernelNotResponding[0m: The kernel died unexpectedly and has been restarted. If the restart fails, restart the kernel from the Kernel menu.

### Finetuning модели на нагенерированных данных:

In [21]:
#!g1.1
import sacrebleu
from datasets import load_metric 

metric = load_metric("sacrebleu")

#def compute_metrics(rsl_preds, truth_rsl):
#    # TODO: also implement BERT-score
#    rus_rsl_bleu = sacrebleu.corpus_bleu(rsl_preds, [truth_rsl])
#    return {"BLEU": rus_rsl_bleu.score}

In [ ]:
#!g1.1
model = RusRSLModel()

In [22]:
#!g1.1
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

batch_size = 64
logging_steps = train_dataset_encoded['input_ids'].shape[0] // batch_size
model_name = f"modelT5-finetuned-gen"

args = Seq2SeqTrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_steps=logging_steps,
    fp16=False)  # True
#push_to_hub=True,

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [23]:
#!g1.1
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [24]:
#!g1.1
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset_encoded,
    eval_dataset=test_dataset_encoded,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

KeyError: 'Indexing with integers (to access backend Encoding for a given batch index) is not available when using Python based tokenizers'

***** Running training *****
  Num examples = 3
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 3
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: annaklezovich (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:872: UserWarning: The following variables cannot be serialized: trainer
  warnings.warn(message)
